In [576]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.plotting import scatter_matrix
from sklearn import svm, feature_selection, linear_model
from sklearn.model_selection import train_test_split
import statsmodels.formula.api as smf
import statsmodels.api as sm

In [577]:
def markOutliers(df,columns):

    t = np.zeros(df.shape[0])
    z = np.zeros(df.shape[0])
    for column in columns:
        mean=np.mean(df[column])
        sd=np.std(df[column])
        threshold=2
        test=[]

        for i, x in enumerate(df[column]):
            if column=="SALE_PRICE":
                if x<10000:
                    t[i]=1
            z[i]=(x-mean)/sd
            if z[i]>=threshold:
                test.append((x,z[i]))
                t[i]=1
        print(column,'\n', test)
    df['outlier']=t
    
    return df

In [578]:
def step2_clean_1():
    df = pd.read_csv("Manhattan12.csv")

    # Print shape
    print(df.shape)

    # Rename incorrect column names
    df.rename(columns={"APART\r\nMENT\r\nNUMBER":"APARTMENT NUMBER", "SALE\r\nPRICE":"SALE PRICE"}, inplace = True)
    numerical=['RESIDENTIAL_UNITS','COMMERCIAL_UNITS','TOTAL_UNITS','LAND_SQUARE_FEET','GROSS_SQUARE_FEET','SALE_PRICE']
    categorical=['BOROUGH','NEIGHBORHOOD', 'BUILDING_CLASS_CATEGORY', 'TAX_CLASS_AT_PRESENT', 'BLOCK','LOT','EASE-MENT', 'BUILDING_CLASS_AT_PRESENT', 'ADDRESS', 'APARTMENT_NUMBER','ZIP_CODE','YEAR_BUILT','TAX_CLASS_AT_TIME_OF_SALE', 'BUILDING_CLASS_AT_TIME_OF_SALE', 'SALE_DATE'
    ]
    df.columns=df.columns.str.replace(' ', '_')
    # df_num_man=df.filter(['RESIDENTIAL_UNITS', 'COMMERCIAL_UNITS', 'TOTAL_UNITS', 'LAND SQUARE FEET', 'GROSS SQUARE FEET', 'SALE PRICE']).copy()

    #num_cols=pd.concat([num_cols,df_num_man], axis=1, join='inner')
    df[numerical]=df[numerical].replace('\$','', regex=True)
    df[numerical]=df[numerical].replace(',','', regex=True)


    df['SALE_DATE']=pd.to_datetime(df['SALE_DATE'], dayfirst=True)
    df[categorical]=df[categorical].replace(' ', '', regex=True)
    df[categorical]=df[categorical].replace('', np.NaN)

    df[numerical]=df[numerical].apply(pd.to_numeric)
    df[numerical]=df[numerical].replace(0, np.NaN)
    df.dropna(subset=["NEIGHBORHOOD"], inplace=True)



    df.drop(columns=['BOROUGH', 'EASE-MENT', 'APARTMENT_NUMBER'], inplace=True)
    df=markOutliers(df, numerical)
    df=df[df.outlier==0]
    df=df.drop('outlier', axis=1)
    df.reset_index(drop=True, inplace=True)

    return df

In [579]:
def convertNeighborhood(df):
    m = np.zeros(df.shape[0])
    dfgroup=df.groupby(by="NEIGHBORHOOD", as_index=False).mean()
    #df.groupby(by="NEIGHBORHOOD", as_index=False).mean().to_csv('groupednorm.csv')
    print(dfgroup)
    
    n=np.zeros(dfgroup.shape[0])
    for i, r in dfgroup.iterrows():
        if r["SALE_PRICE"]<.12:
            n[i]=1
        elif r["SALE_PRICE"]>.15:
            n[i]=3
        else:
            n[i]=2
    dfgroup['NCat']=n
    for i, r in df.iterrows():
        temp = dfgroup[dfgroup["NEIGHBORHOOD"]==r["NEIGHBORHOOD"]]["NCat"]
        m[i]=temp

    return m

def normalize(df,num_cols):
    df_norm=df.copy()
    df_norm[num_cols]=df_norm[num_cols]/df_norm[num_cols].abs().max()
    return df_norm

In [586]:
def checkUnits(df):
    t = np.zeros(df.shape[0])
    for i, r in df.iterrows():
        if pd.isnull(r["RESIDENTIAL_UNITS"]) and pd.isnull(r["COMMERCIAL_UNITS"]) and pd.isnull(r["TOTAL_UNITS"]):
            t[i]=1
    df['no_units']=t
    return df

def fillUnits(df):
    count=0
    for i, r in df.iterrows():
        if r["RESIDENTIAL_UNITS"]+r["COMMERCIAL_UNITS"]!=r["TOTAL_UNITS"]:
            r["TOTAL_UNITS"]=r["RESIDENTIAL_UNITS"]+r["COMMERCIAL_UNITS"]
            if r["RESIDENTIAL_UNITS"]==0 and r["COMMERCIAL_UNITS"]!=0:
                print(r["TOTAL_UNITS"]-r["COMMERCIAL_UNITS"])
            if r["RESIDENTIAL_UNITS"]!=0 and r["COMMERCIAL_UNITS"]==0:
                print("com 0")
    return df["TOTAL_UNITS"]



In [581]:
numerical=['RESIDENTIAL_UNITS','COMMERCIAL_UNITS','TOTAL_UNITS','LAND_SQUARE_FEET','GROSS_SQUARE_FEET','SALE_PRICE']
categorical=['BOROUGH','NEIGHBORHOOD', 'BUILDING_CLASS_CATEGORY', 'TAX_CLASS_AT_PRESENT', 'BLOCK','LOT','EASE-MENT', 'BUILDING_CLASS_AT_PRESENT', 'ADDRESS', 'APARTMENT_NUMBER','ZIP_CODE','YEAR_BUILT','TAX_CLASS_AT_TIME_OF_SALE', 'BUILDING_CLASS_AT_TIME_OF_SALE', 'SALE_DATE']
df=normalize(step2_clean_1(),numerical)
df["NCat"]=convertNeighborhood(df)
print(df.shape)
print(df.isnull().sum())

(27395, 21)
RESIDENTIAL_UNITS 
 [(61.0, 2.4338152371257764), (59.0, 2.3478298863349667), (68.0, 2.7347639648936113), (134.0, 5.572280540990339), (162.0, 6.7760754520616775), (180.0, 7.549943609178967), (133.0, 5.529287865594934), (65.0, 2.6057859387073963), (56.0, 2.2188518601487517), (56.0, 2.2188518601487517), (538.0, 22.941321400733937), (66.0, 2.648778614102801), (66.0, 2.648778614102801), (255.0, 10.774394263834338), (157.0, 6.561112075084653), (91.0, 3.723595498987925), (55.0, 2.175859184753347), (70.0, 2.820749315684421), (54.0, 2.132866509357942), (92.0, 3.76658817438333), (85.0, 3.465639446615495), (52.0, 2.046881158567132), (90.0, 3.68060282359252), (85.0, 3.465639446615495), (85.0, 3.465639446615495), (117.0, 4.841405059268454), (123.0, 5.099361111640883), (84.0, 3.4226467712200903), (522.0, 22.25343859440746), (63.0, 2.5198005879165866), (89.0, 3.637610148197115), (59.0, 2.3478298863349667), (53.0, 2.0898738339625367), (62.0, 2.4768079125211813), (62.0, 2.4768079125211813),

C:\Users\zacha\AppData\Local\Temp\ipykernel_33468\2447460760.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  dfgroup=df.groupby(by="NEIGHBORHOOD", as_index=False).mean()


(26448, 19)
NEIGHBORHOOD                          0
BUILDING_CLASS_CATEGORY            3101
TAX_CLASS_AT_PRESENT                125
BLOCK                                 0
LOT                                   0
BUILDING_CLASS_AT_PRESENT           125
ADDRESS                               0
ZIP_CODE                              0
RESIDENTIAL_UNITS                 15730
COMMERCIAL_UNITS                  23242
TOTAL_UNITS                        9652
LAND_SQUARE_FEET                  22316
GROSS_SQUARE_FEET                 22470
YEAR_BUILT                            0
TAX_CLASS_AT_TIME_OF_SALE             0
BUILDING_CLASS_AT_TIME_OF_SALE        0
SALE_PRICE                         7510
SALE_DATE                             0
NCat                                  0
dtype: int64


In [582]:
df=checkUnits(df)
df=df[df.no_units==0]
df=df.drop('no_units', axis=1)
df.reset_index(drop=True, inplace=True)
print(df.shape)
print(df.isnull().sum())

(16796, 19)
NEIGHBORHOOD                          0
BUILDING_CLASS_CATEGORY            3081
TAX_CLASS_AT_PRESENT                  0
BLOCK                                 0
LOT                                   0
BUILDING_CLASS_AT_PRESENT             0
ADDRESS                               0
ZIP_CODE                              0
RESIDENTIAL_UNITS                  6078
COMMERCIAL_UNITS                  13590
TOTAL_UNITS                           0
LAND_SQUARE_FEET                  12828
GROSS_SQUARE_FEET                 12825
YEAR_BUILT                            0
TAX_CLASS_AT_TIME_OF_SALE             0
BUILDING_CLASS_AT_TIME_OF_SALE        0
SALE_PRICE                         6622
SALE_DATE                             0
NCat                                  0
dtype: int64


In [585]:


df[numerical]=df[numerical].replace(np.NaN, 0)
df["TOTAL_UNITS"]=fillUnits(df)
df[numerical]=df[numerical].replace(0,np.NaN)
print(df.shape)
print(df.isnull().sum())

Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 0
Res 

In [584]:
def cleanSquareFt(df):
    dfIncomplete=df[["LAND_SQUARE_FEET","GROSS_SQUARE_FEET"]].copy()
    dfComplete=df[["LAND_SQUARE_FEET","GROSS_SQUARE_FEET","SALE_PRICE","RESIDENTIAL_UNITS", "COMMERCIAL_UNITS", "TOTAL_UNITS","NCat"]].copy()
    dfComplete[["LAND_SQUARE_FEET","GROSS_SQUARE_FEET","SALE_PRICE"]].replace(0,np.NaN)
    dfComplete.dropna(inplace=True)


    # Define feature matrix X and target variable y
    X = dfComplete[["LAND_SQUARE_FEET","SALE_PRICE","RESIDENTIAL_UNITS", "COMMERCIAL_UNITS", "TOTAL_UNITS","NCat"]]
    y = dfComplete["GROSS_SQUARE_FEET"]

    # Split data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

    # Train a linear regression model
    model = linear_model.LinearRegression()
    model.fit(X_train, y_train)

    # Evaluate the model on the testing data
    score = model.score(X_test, y_test)
    print("R^2 score:", score)

In [575]:
cleanSquareFt(df)

R^2 score: 0.7116178096914778
